# <center> 🧽 Data Cleaning Part 2 </center>

## Table of Contents

- [1. Introduction](#1-introduction)
- [2. Loading Data](#2-loading-data)

## 1. Introduction
[Back to Table of Contents](#table-of-contents)

---

| ⚡ Description: Introducing the project ⚡ |
| :--------------------------- |

---

<a id="two"></a>
## 2. Loading Data
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data in our environment ⚡ |
| :--------------------------- |

---

### Import packages

In [1]:
# Import packages
import re

import numpy as np
import pandas as pd

In [ ]:
listings_for_sale = pd.read_csv("../data/raw/listings_for_sale_in_capital_cities.csv")
display("Listings for sale", listings_for_sale.head(), listings_for_sale.shape)

In [ ]:
# Get information about the listings_for_sale document
listings_for_sale.info()

In [ ]:
# See the number of null values in each column
listings_for_sale.isnull().sum()

In [ ]:
# Drop listings where the property_title is null
listings_for_sale = listings_for_sale.dropna(subset=['property_title']).reset_index(drop=True)

In [ ]:
listings_for_sale_duplicates = listings_for_sale[listings_for_sale.duplicated()]
num_listings_for_sale_duplicates = len(listings_for_sale_duplicates)
print(f"There are {num_listings_for_sale_duplicates} duplicates in the listings_for_sale dataset ")

In [ ]:
# Drop duplicate rows and keep the first occurrence
listings_for_sale = listings_for_sale.drop_duplicates().reset_index(drop=True)

In [ ]:
listings_for_sale['price_cleaned'] = listings_for_sale['price'].str.extract(
    r'(R\s?\d[\d\s]*)')

listings_for_sale['price_cleaned'] = listings_for_sale['price_cleaned'].str.replace(
    r'\s+', '', regex=True)

listings_for_sale['price (in rands)'] = listings_for_sale['price_cleaned'].str.extract(
    r'R?(\d+)', expand=False)

# Print out the first five rows to see the results
listings_for_sale.head()

In [ ]:
# Extract the location from the listing_slug column
listings_for_sale['suburb'] = listings_for_sale['property_title'].str.extract(
    r'for Sale in (.+)', expand=False)

# Remove the trailing numbers from the suburb column
listings_for_sale['suburb'] = (
    listings_for_sale['suburb'].str.replace(r'-\d+$', '', regex=True).str.strip())

# Print the first five rows of the table to see the results
listings_for_sale.head()

In [ ]:
def extract_city_from_location(location, city_list):
    if pd.isna(location):
        return None
    
    
    location_lower = location.lower()
    for city in city_list:
        if city.lower() in location_lower:
            return city
    return None

In [ ]:
# Apply the extract_city_from_location function to extract the function
listings_for_sale['city'] = listings_for_sale['location'].apply(
    lambda loc: extract_city_from_location(loc, city_list))

In [ ]:
listings_for_sale['city'].isnull().sum()

In [ ]:
# Create a mapping of suburb to kmown cities
suburb_to_city_map = listings_for_sale[listings_for_sale['city'].notna()].drop_duplicates(
    subset=['suburb', 'city']).set_index('suburb')['city'].to_dict()

# Fill missing city values based on the suburb
listings_for_sale['city'] = listings_for_sale.apply(lambda row: suburb_to_city_map.get(
    row['suburb'], row['city']) if pd.isna(row['city']) else row['city'], axis=1)

In [ ]:
# See the number of unique values in the city column
listings_for_sale['city'].value_counts()

In [ ]:
listings_for_sale.isnull().sum()

In [ ]:
# Drop the rows where the city_column is empty
listings_for_sale = listings_for_sale.dropna(subset=['city'])

In [ ]:
# Merge to get the matching province for the listings
listings_for_sale = listings_for_sale.merge(city_province, on='city', how='left')

In [ ]:
poa_mask = listings_for_sale['price'].astype(
    str).str.contains("POA", case=False, na=False)
num_poa = poa_mask.sum()

print(f"There are {num_poa} listings with 'POA' in the price column.")